In [40]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px solid gray;
}

# Weightlifting Data Retrieval and Preparation

import the csv into dataframe

In [41]:
import pandas as pd

In [42]:
df = pd.read_csv('olympics_weightlifting_1996_to_2020.csv',encoding='utf-8-sig')
df

,Rank,Athlete,Group,Bodyweight,1,2,3,Result,1.1,2.1,3.1,Result.1,Total,Year,Nation
0,1.0,Halil Mutlu (TUR),A,53.91,125.0,130.0,132.5,132.5,152.5,152.5,155.0,155.0,287.5,1996,NaN
1,2.0,Zhang Xiangsen (CHN),A,53.39,122.5,127.5,130.0,130.0,150.0,155.0,157.5,150.0,280.0,1996,NaN
2,3.0,Sevdalin Minchev (BUL),A,54.0,117.5,122.5,125.0,125.0,147.5,152.5,157.5,152.5,277.5,1996,NaN
3,4.0,Lan Shizhang (CHN),A,53.61,120.0,125.0,127.5,125.0,150.0,157.5,162.5,150.0,275.0,1996,NaN
4,5.0,Traian Cihărean (ROU),A,53.9,115.0,120.0,122.5,120.0,140.0,145.0,152.5,145.0,265.0,1996,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1694,10,Sarah Fischer,A,93.35,93,97,97,97,117,123,123,123,220,2020,Austria
1695,11,Anna Van Bellinghen,B,87.1,96,100,100,96,115,119,123,123,219,2020,Belgium
1696,12,Erdenebatyn Bilegsaikhan,B,NaN,80,85,87,85,115,120,122,122,207,2020,Mongolia
1697,13,Scarleth Ucelo,B,113.5,86,87,87,87,107,112,116,116,203,2020,Guatemala


firstly, remove the unnecessary columns and rename columns to relevant names.

In [43]:
df.columns

Index(['Rank', 'Athlete', 'Group', 'Bodyweight', '1', '2', '3', 'Result',
       '1.1', '2.1', '3.1', 'Result.1', 'Total', 'Year', 'Nation'],
      dtype='object')

In [44]:
df.drop(columns=['1','2','3','1.1','2.1','3.1','Group'],inplace=True)
df.rename(columns={'Result':'Snatch','Result.1':'Clean & Jerk'},inplace=True)
df

,Rank,Athlete,Bodyweight,Snatch,Clean & Jerk,Total,Year,Nation
0,1.0,Halil Mutlu (TUR),53.91,132.5,155.0,287.5,1996,NaN
1,2.0,Zhang Xiangsen (CHN),53.39,130.0,150.0,280.0,1996,NaN
2,3.0,Sevdalin Minchev (BUL),54.0,125.0,152.5,277.5,1996,NaN
3,4.0,Lan Shizhang (CHN),53.61,125.0,150.0,275.0,1996,NaN
4,5.0,Traian Cihărean (ROU),53.9,120.0,145.0,265.0,1996,NaN
...,...,...,...,...,...,...,...,...
1694,10,Sarah Fischer,93.35,97,123,220,2020,Austria
1695,11,Anna Van Bellinghen,87.1,96,123,219,2020,Belgium
1696,12,Erdenebatyn Bilegsaikhan,NaN,85,122,207,2020,Mongolia
1697,13,Scarleth Ucelo,113.5,87,116,203,2020,Guatemala


now we see a lot of NaN in the nations column. Lets fix that.

First, we grab the string in brackets in the athlete column which indicate an IOC code. Some rows contain this so we will use np.where to determine what to do if the athlete has an IOC code.

In [45]:
import numpy as np
df['Nation'] = np.where(df['Athlete'].str.contains('\('),df['Athlete'].str[-5:],df['Nation'])
df['Nation']

0             (TUR)
1             (CHN)
2             (BUL)
3             (CHN)
4             (ROU)
           ...     
1694        Austria
1695        Belgium
1696       Mongolia
1697      Guatemala
1698    New Zealand
Name: Nation, Length: 1699, dtype: object

As we have extracted the IOC code from the athlete's name, we should now remove the brackets inside their name. Also, we can remove the brackets from the IOC codes in the Nations column too.


In [46]:
df['Nation'] = df['Nation'].str.strip('\(\)')
df['Athlete'] = df['Athlete'].str.split('\(').str[0]
df.head(5)

,Rank,Athlete,Bodyweight,Snatch,Clean & Jerk,Total,Year,Nation
0,1.0,Halil Mutlu,53.91,132.5,155.0,287.5,1996,TUR
1,2.0,Zhang Xiangsen,53.39,130.0,150.0,280.0,1996,CHN
2,3.0,Sevdalin Minchev,54.0,125.0,152.5,277.5,1996,BUL
3,4.0,Lan Shizhang,53.61,125.0,150.0,275.0,1996,CHN
4,5.0,Traian Cihărean,53.9,120.0,145.0,265.0,1996,ROU


Now lets map each IOC to their corresponding Nation name. We will do this with the IOC table from Wikipedia. Make the IOC table a dictionary and use the map function to map series values based on key:value.

In [47]:
from bs4 import BeautifulSoup
import requests

In [48]:
# map ioc to country name
ioc_codes_url = 'https://en.wikipedia.org/wiki/List_of_IOC_country_codes'
soup = BeautifulSoup(requests.get(ioc_codes_url).content,'html.parser')
#remove all references, such as [3] or [5]
for tag in soup.find_all(class_='reference'):
    tag.decompose()
        
ioc = pd.read_html(str(soup))[0]

ioc['Code'] = ioc['Code'].str[-3:]
ioc = ioc.set_index('Code')['National Olympic Committee']
ioc

Code
AFG                         Afghanistan
ALB                             Albania
ALG                             Algeria
AND                             Andorra
ANG                              Angola
                     ...               
VIE                             Vietnam
VIN    Saint Vincent and the Grenadines
YEM                               Yemen
ZAM                              Zambia
ZIM                            Zimbabwe
Name: National Olympic Committee, Length: 206, dtype: object

Now that we have the IOC dictionary, use map on the series with string length of 3, as IOC codes all have length of 3.

In [49]:
nation = df['Nation'][df['Nation'].str.len() == 3].map(ioc)
nation

0               Turkey
1                China
2             Bulgaria
3                China
4              Romania
             ...      
1398    Chinese Taipei
1399             Egypt
1400      Turkmenistan
1578               NaN
1616               NaN
Name: Nation, Length: 1367, dtype: object

Now that we have a subset of the Nations with the correct nation name, we now update the existing Nation column with these new values

In [50]:
df['Nation'].update(nation)
df

,Rank,Athlete,Bodyweight,Snatch,Clean & Jerk,Total,Year,Nation
0,1.0,Halil Mutlu,53.91,132.5,155.0,287.5,1996,Turkey
1,2.0,Zhang Xiangsen,53.39,130.0,150.0,280.0,1996,China
2,3.0,Sevdalin Minchev,54.0,125.0,152.5,277.5,1996,Bulgaria
3,4.0,Lan Shizhang,53.61,125.0,150.0,275.0,1996,China
4,5.0,Traian Cihărean,53.9,120.0,145.0,265.0,1996,Romania
...,...,...,...,...,...,...,...,...
1694,10,Sarah Fischer,93.35,97,123,220,2020,Austria
1695,11,Anna Van Bellinghen,87.1,96,123,219,2020,Belgium
1696,12,Erdenebatyn Bilegsaikhan,NaN,85,122,207,2020,Mongolia
1697,13,Scarleth Ucelo,113.5,87,116,203,2020,Guatemala


In [51]:
df['Nation'].value_counts()

China          68
South Korea    56
Belarus        51
Ukraine        51
Turkey         47
               ..
Comoros         1
Guyana          1
Guam            1
IOA             1
Lebanon         1
Name: Nation, Length: 131, dtype: int64

Another thing to fix is Olympic and World records. These records have "OR" or "WR" with the value. This should be removed.


In [52]:
df.iloc[1603]

Rank                   1.0
Athlete         Hou Zhihui
Bodyweight            49.0
Snatch               94 OR
Clean & Jerk        116 OR
Total               210 OR
Year                  2020
Nation               China
Name: 1603, dtype: object

In [53]:
columns_to_remove_characters = ['Snatch',
         'Clean & Jerk',
         'Total']
for c in columns_to_remove_characters:
    df[c].replace(r'[^0-9^.]','',regex=True,inplace=True)
    
df.iloc[1603]

Rank                   1.0
Athlete         Hou Zhihui
Bodyweight            49.0
Snatch                  94
Clean & Jerk           116
Total                  210
Year                  2020
Nation               China
Name: 1603, dtype: object

There is also DQ and DSQ values in the rank column where athletes are disqualified. This will be change to NaN.

In [54]:
df['Rank']

0       1.0
1       2.0
2       3.0
3       4.0
4       5.0
       ... 
1694     10
1695     11
1696     12
1697     13
1698      –
Name: Rank, Length: 1699, dtype: object

In [55]:
# df['Rank'].replace(r'[^0-9]',np.NaN,regex=True,inplace=True)

df['Rank'] = pd.to_numeric(df.Rank.astype(str).str.replace(',',''),errors='coerce')


Another issue is '-' values that are found within the tables, perhaps they are failed attempts / unrecorded values. We will set these to nan

In [56]:
df.replace("–",np.NaN,inplace=True)

In [57]:
df

,Rank,Athlete,Bodyweight,Snatch,Clean & Jerk,Total,Year,Nation
0,1.0,Halil Mutlu,53.91,132.5,155.0,287.5,1996,Turkey
1,2.0,Zhang Xiangsen,53.39,130.0,150.0,280.0,1996,China
2,3.0,Sevdalin Minchev,54.0,125.0,152.5,277.5,1996,Bulgaria
3,4.0,Lan Shizhang,53.61,125.0,150.0,275.0,1996,China
4,5.0,Traian Cihărean,53.9,120.0,145.0,265.0,1996,Romania
...,...,...,...,...,...,...,...,...
1694,10.0,Sarah Fischer,93.35,97,123,220,2020,Austria
1695,11.0,Anna Van Bellinghen,87.1,96,123,219,2020,Belgium
1696,12.0,Erdenebatyn Bilegsaikhan,NaN,85,122,207,2020,Mongolia
1697,13.0,Scarleth Ucelo,113.5,87,116,203,2020,Guatemala


Now we can arrange the column orders, I would like Nation to be near the top 3.

Now that everything is sorted, we will clean na values.

In [58]:
df.isna().sum()

Rank            296
Athlete           0
Bodyweight        6
Snatch            0
Clean & Jerk      3
Total             0
Year              0
Nation            0
dtype: int64

In [59]:
df.dropna(inplace=True)

To make things pretty, we can reorganise the columns.

In [60]:
#rearrange columns
df = df[['Rank','Athlete','Nation','Bodyweight','Snatch','Clean & Jerk','Total','Year']]

In [61]:
df.head(10)

,Rank,Athlete,Nation,Bodyweight,Snatch,Clean & Jerk,Total,Year
0,1.0,Halil Mutlu,Turkey,53.91,132.5,155.0,287.5,1996
1,2.0,Zhang Xiangsen,China,53.39,130.0,150.0,280.0,1996
2,3.0,Sevdalin Minchev,Bulgaria,54.0,125.0,152.5,277.5,1996
3,4.0,Lan Shizhang,China,53.61,125.0,150.0,275.0,1996
4,5.0,Traian Cihărean,Romania,53.9,120.0,145.0,265.0,1996
5,6.0,Ivan Ivanov,Bulgaria,53.9,112.5,145.0,257.5,1996
6,7.0,Ko Kwang-ku,South Korea,53.89,115.0,140.0,255.0,1996
7,8.0,Juan Fernández,Colombia,53.94,110.0,145.0,255.0,1996
8,9.0,Wang Shin-yuan,Chinese Taipei,53.56,105.0,145.0,250.0,1996
9,10.0,Toshiyuki Notomi,Japan,53.6,110.0,140.0,250.0,1996


In [62]:
df.tail(10)

,Rank,Athlete,Nation,Bodyweight,Snatch,Clean & Jerk,Total,Year
1687,3.0,Sarah Robles,United States,148.3,128,154,282,2020
1688,4.0,Lee Seon-mi,South Korea,118.9,125,152,277,2020
1689,5.0,Nurul Akmal,Indonesia,113.55,115,141,256,2020
1690,6.0,Charisma Amoe-Tarrant,Australia,154.05,105,138,243,2020
1691,7.0,Verónica Saladín,Dominican Republic,126.2,111,131,242,2020
1692,8.0,Kuinini Manumua,Tonga,108.5,103,125,228,2020
1693,9.0,Eyurkenia Duverger,Cuba,103.65,96,129,225,2020
1694,10.0,Sarah Fischer,Austria,93.35,97,123,220,2020
1695,11.0,Anna Van Bellinghen,Belgium,87.1,96,123,219,2020
1697,13.0,Scarleth Ucelo,Guatemala,113.5,87,116,203,2020


Before exporting to a csv, lets clean up the rank column to be as type int

In [63]:
df['Rank'] = df['Rank'].astype(int)

In [64]:
df.head(10)

,Rank,Athlete,Nation,Bodyweight,Snatch,Clean & Jerk,Total,Year
0,1,Halil Mutlu,Turkey,53.91,132.5,155.0,287.5,1996
1,2,Zhang Xiangsen,China,53.39,130.0,150.0,280.0,1996
2,3,Sevdalin Minchev,Bulgaria,54.0,125.0,152.5,277.5,1996
3,4,Lan Shizhang,China,53.61,125.0,150.0,275.0,1996
4,5,Traian Cihărean,Romania,53.9,120.0,145.0,265.0,1996
5,6,Ivan Ivanov,Bulgaria,53.9,112.5,145.0,257.5,1996
6,7,Ko Kwang-ku,South Korea,53.89,115.0,140.0,255.0,1996
7,8,Juan Fernández,Colombia,53.94,110.0,145.0,255.0,1996
8,9,Wang Shin-yuan,Chinese Taipei,53.56,105.0,145.0,250.0,1996
9,10,Toshiyuki Notomi,Japan,53.6,110.0,140.0,250.0,1996


In [65]:
df.to_csv('Cleaned_Weightlifting_1996_to_2020.csv',encoding='utf-8-sig',index=False)